In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
url = "https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find(class_ = 'socrata-visualization')
element = driver.find_element_by_class_name('pager-button-next')
element.click()

In [ ]:
data = pd.DataFrame([[td.text.strip() for td in row.findAll('td')] for row in table.tbody.findAll('tr')], \
                       columns= [x.text.strip() for x in table.find_all(class_ = 'column-header-content-column-name')])
click = 0
for i in range(10000):
    element = driver.find_element_by_class_name('pager-button-next')
    element.click()
    click += 1
    
    #driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find(class_ = 'socrata-visualization')
    
    time.sleep(2)
    
    data = data.append(pd.DataFrame([[td.text.strip() for td in row.findAll('td')] for row in table.tbody.findAll('tr')],\
                                    columns= [x.text.strip() for x in table.find_all(class_ = 'column-header-content-column-name')])).reset_index(drop = True)
    #to make sure that data is saved if process crashes
    data.to_csv('car_crashes.csv')

In [ ]:
data = pd.read_csv('car_crashes.csv')

In [ ]:
from datetime import datetime, timedelta
from urllib.request import urlopen

In [ ]:
driver = webdriver.Chrome(chromedriver)

In [ ]:
def scrape_weather(begin_date, end_date):
    current_date = datetime.strptime(begin_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')+timedelta(days=1) #add one to make loop end on the end date
    
    lookup_URL = 'https://www.wunderground.com/history/daily/us/il/chicago/KMDW/date/{}-{}-{}'
    
    
    result = None
    
    name_of_csv = '{}new.csv'.format(begin_date)
    
    while current_date != end_date:
        formatted_lookup_URL = lookup_URL.format(current_date.year, current_date.month, current_date.day)
        #print(formatted_lookup_URL)
        
        driver.get(formatted_lookup_URL)
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'history-observation-table')))
        except TimeoutException:
            print("Loading took too much time! Trying one more time:")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table', {'id': 'history-observation-table'})
        
        df = pd.DataFrame([[td.text.strip() for td in row.findAll('td')] for row in table.tbody.findAll('tr')])
        df['Date'] = current_date
        
        if result is None:
            result = df
        else:
            result = result.append(df).reset_index(drop = True)
        
        result.to_csv(name_of_csv)
        current_date += timedelta(days=1)
    
    return result 